### Import libraries

In [4]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

c:\Users\david\anaconda3\envs\DL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### Load Data

In [5]:
# Load the JSON file into a DataFrame
file_path = 'data/sarcasm_data.json'
df = pd.read_json(file_path).transpose()

# Reset the index to turn the first element into a new column
df = df.reset_index()

### Hartman sentiments context overall

In [15]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
def hartmann_sentiment(text):
    text = classifier(text)
    # Assuming 'text' is already defined and contains the necessary data
    sentiments = [text[0][i]['score'] for i in range(6)]
    sentiment = np.argmax(sentiments)
    return sentiment

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [16]:
# Initialize data frame
df_h = pd.DataFrame()

# Apply sentiment analysis to the 'utterance' column
df_h['sentiment_utterance'] = df['utterance'].apply(hartmann_sentiment)

# Apply sentiment analysis to the 'context' column
df_h['sentiment_context_all'] = df['context'].apply(hartmann_sentiment)

df_h

# anger = 0, disgust = 1, fear = 2,  joy = 3, neutral = 4, sadness =  5, surprise = 6

,sentiment_utterance,sentiment_context_all
0,4,4
1,4,4
2,4,4
3,4,4
4,4,2
...,...,...
685,0,0
686,1,0
687,4,4
688,1,4


### Hartmann sentiment 3 context sentences + utterance

In [23]:
# Apply sentiment analysis to each sentence in the 'context' column
sentiment_context_per_sentence = df['context'].apply(lambda context: [hartmann_sentiment(sentence) for sentence in context])

# inialize array
X_sentences = []

# Fill array with sentiments
for idx, sentence in enumerate(sentiment_context_per_sentence):
    # Ensure each list has at least 3 context sequences
    sentence.extend([4] * (3 - len(sentence)))# Classify as 4 (Neutral) if no context sentence
    sentence = sentence[:3]
    X_sentences.append(sentence)

# Format dataframe
columns = ["Ctx_1", "Ctx_2", "Ctx_3"]
df_h_3 = pd.DataFrame(X_sentences, columns=columns)
df_h_3["Utt"] = df_h['sentiment_utterance']

# Display dataframe
df_h_3

# anger = 0, disgust = 1, fear = 2,  joy = 3, neutral = 4, sadness =  5, surprise = 6, no sentence = 4

,Ctx_1,Ctx_2,Ctx_3,Utt
0,4,5,4,4
1,4,4,4,4
2,4,4,4,4
3,4,0,4,4
4,2,1,1,4
...,...,...,...,...
685,0,4,4,0
686,0,4,1,1
687,4,0,4,4
688,4,4,3,1
